In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

## 留出法

In [ ]:
num_validation_samples = 10000
# 一般都要打乱数据，不然验证集的顺序会影响预测结果
np.random.shuffle(data)
validation_data = data[:num_validation_samples]
data = data[num_validation_samples:]
training_data = data[:]
model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data)
# ……
# 如果在验证集上的预测结果也比较好，就使用所有数据进行训练
model = get_model()
model.train(np.concatenate([training_data, validation_data]))
# 最后，在测试集上进行预测
test_score = model.evaluate(test_data)

## 留出法不适用于数据少的情况，故有K折交叉验证

In [ ]:
k = 4
num_validation_samples = len(data) // k
# 一般都会打乱数据
np.random.shuffle(data)
validation_scores = []
for fold in range(k):
    validation_data = data[num_validation_samples * fold:
                           num_validation_samples * (fold + 1)]
    training_data = data[:num_validation_samples * fold] + data[num_validation_samples * (fold + 1):]
    model = get_model()
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)
validation_score = np.average(validation_scores)
model = get_model()
model.train(data)
test_score = model.evaluate(test_data)

## 带有打乱数据的重复k折交叉验证：重复P次K折交叉验证，每次重复都打乱数据
## 对于分类问题，数据都要打乱；对于时间序列问题，数据不要打乱
## 确保训练集和验证集没有交集

# 数据预处理
## 向量化
都要是浮点数张量（特定情况可以是整数张量）
## 标准化
最终使得均值为0，标准差为1
此处为z-score标准化
x -= x.mean(axis=0) # 均值为0
std方法计算标准差
x /= x.std(axis=0) # 标准差为1
## 缺失值处理
一般设置为0（前提是0为无意义值），这样网络会自动忽略这些值

## 特征工程
对数据进行预处理，提取适合网络训练的特征

## 确定网络模型最佳层数或每层最佳大小

In [ ]:
# 要找到合适的模型大小，一般的工作流程是开始时选择相对较少的层和参数，然后逐渐增加层的大小或增加新层，直到这种增加对验证损失的影响变得很小

from tensorflow.keras import models, layers


model = models.Sequential()
# 每层16个神经元
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# 下次实验用4个神经元，画图两次val_loss，判断哪个更合适

## 奥卡姆剃刀（Occam’s razor）原理：如果一件事情有两种解释，那么最可能正确的解释就是最简单的那个，即假设更少的那个。
## 深度学习中，简单模型指参数值分布的熵更小的模型或参数更少的模型

In [7]:
# L1正则化：添加的成本与权重系数的绝对值成正比
# L2正则化（权重衰减）：添加的成本与权重系数的平方（权重的 L2 范数）成正比
# 正则化避免过拟合
from tensorflow.keras import regularizers, models, layers
from tensorflow.keras.optimizers import RMSprop
model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
 activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
 activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# dropout正则化，每一层出来都随机置0
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
# 随机将50%的置0
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout_8 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_9 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


## 常用激活函数和损失函数场景

|      问题类型      |  最后一层激活  |            损失函数            |
|:--------------:|:--------:|:--------------------------:|
|     二分类问题      | sigmoid  |    binary_crossentropy     |
|   多分类、单标签问题    | softmax  |  categorical_crossentropy  |
 |   多分类、多标签问题    | sigmoid  |    binary_crossentropy     |
 |     回归到任意值     |    无     |            mse             |
| 回归到 0~1 范围内的值  | sigmoid  | mse 或 binary_crossentropy  |